In [ ]:
import sys
from bert import QA
import gzip
from tqdm import tqdm
import json
from fuzzywuzzy import fuzz
import pandas as pd
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_colwidth', None)
from bs4 import BeautifulSoup
import sys
sys.path.insert(1, '../timelywimely/')
import wikiapi
from wikiapi import get_wikipedia_revision
import requests
import datetime
import numpy as np

In [ ]:
from datetime import date

In [ ]:
i = 0
list_of_q = []
# with gzip.open('../tiny-dev/nq-dev-sample.jsonl.gz') as f:
# with gzip.open('../tiny-dev/v1.0-simplified_nq-dev-all.jsonl.gz') as f:
with gzip.open('../tiny-dev/v1.0-simplified_simplified-nq-train.jsonl.gz') as f: 
    for line in tqdm(f):
        list_of_q.append(line)
        i += 1

In [ ]:
x = json.loads(list_of_q[1])
start_token = x['annotations'][0]['long_answer']['start_token']
end_token = x['annotations'][0]['long_answer']['end_token']
" ".join(x["document_text"].split(" ")[start_token:end_token])

In [ ]:
x = json.loads(list_of_q[0])
start_token = x['annotations'][0]['short_answers'][0]['start_token']
end_token = x['annotations'][0]['short_answers'][0]['end_token']
" ".join(x["document_text"].split(" ")[start_token:end_token])


In [ ]:
# " ".join(x["document_text"].split(" "))

In [ ]:
# t = get_paragraphs(x['document_text'])

In [ ]:
model = QA('model')
# model.predict([t,t],[x['question_text'],x['question_text']])

In [ ]:
API_URL = "https://wikipedia.org/w/api.php"
INDEX_URL = "https://wikipedia.org/w/index.php"

def get_wikipedia_revision(title, date, diagnostic=False):
    session = requests.Session()
    parameters = {
        "action": "query",
        "prop": "revisions",
        "titles": title,
        "rvprop": "ids|timestamp|comment",
        "rvslots": "main",
        "rvstart": date.isoformat(),
        "rvlimit": "1",
        "formatversion" : "2",
        "format": "json"
    }
#     print(parameters)
    revision_response = session.get(url=API_URL, params=parameters)
    revision_response.raise_for_status()
    
    # revision data succeeded
    revision_data = revision_response.json()
#     print(revision_data)
    try:
        parameters = {
#             "title" : title,
            "oldid" : revision_data['query']['pages'][0]['revisions'][0]['revid']
        }
    except (IndexError,KeyError) as e:
        if not diagnostic:
            return None, None
        else:
            raise e
#     print(parameters)
    page_response = session.get(url=INDEX_URL, params=parameters)
#     return page_response
    return page_response.text, page_response.url
    
    
    
        

In [ ]:
def get_short_answers_train_set(question):
    answers = []
    for an in question['annotations']:
        if len(an['short_answers']) > 0:
            for sa in an['short_answers']:
                start_token = sa['start_token']
                end_token = sa['end_token']
                answ_text = " ".join(question["document_text"].split(" ")[start_token:end_token])
                answers.append(answ_text)
                
    
    return set(answers)
        
def get_paragraphs(text):
#     print(text)
    if text is not None:
        soup = BeautifulSoup(text)
    # document_text = " ".join(t.text for t in soup.find_all('p'))
        for div in soup.find_all("div", {'class':'mw-revision warningbox'}): 
            div.decompose()
        document_text = " ".join(t.text for t in soup.find_all('p')[:25])
        return document_text
    else: 
        return None
    
def get_long_answers(question):
    answers = []
    for annotation in question['annotations']:
        if len(annotation['long_answer']) != 0:
            short_answer = annotation['long_answer']
#             print(short_answer)
            start_token = short_answer['start_token']
            end_token = short_answer['end_token']
#             print(question['document_tokens'][start_token:end_token])
            a = " ".join(t['token'] for t in question['document_tokens'][start_token:end_token] if not t['html_token'])
            if a != '':
                answers.append(a)
#         else:
#             print('No short answers found in annotation.')
            
    return set(answers)

def get_short_answers(question):
    answers = []
    for annotation in question['annotations']:
        if len(annotation['short_answers']) != 0:
            short_answer = annotation['short_answers'][0]
#             print(short_answer)
            start_token = short_answer['start_token']
            end_token = short_answer['end_token']
#             print(question['document_tokens'][start_token:end_token])
            a = " ".join(t['token'] for t in question['document_tokens'][start_token:end_token])
            answers.append(a)
#         else:
#             print('No short answers found in annotation.')
            
    return set(answers)


def get_all_revs(question, display = False):
    compares = {}
    x = json.loads(question)
#     sa = get_short_answers(x).union(get_long_answers(x))
    sa = get_short_answers_train_set(x)
    if sa == set():
        print("i am sad")
        return [None] * 3
#     print(len(sa), x)

    qtext = x["question_text"]
    title = x["document_url"][x["document_url"].index("title")+6:x["document_url"].index("&amp")]
#     title = x['document_title']
#     dates = [datetime.datetime(2008,1,1),datetime.datetime(2010,1,1),datetime.datetime(2012,1,1),datetime.datetime(2014,1,1),datetime.datetime(2016,1,1),datetime.datetime(2018,1,1), datetime.datetime.now()]
#     dates = [datetime.datetime(2010,1,1),datetime.datetime(2014,1,1),datetime.datetime(2018,1,1), datetime.datetime.now()]
    dates = [datetime.datetime(2008,1,1),datetime.datetime(2012,1,1),datetime.datetime(2016,1,1)]

    ret = ["", qtext]
    col_names = ["GT Answers", "Question"]
    if display:
        print(title)
#     document_html = " ".join(t['token'] for t in x['document_tokens'])
    document_html = " ".join(x["document_text"].split(" "))
    document_text = get_paragraphs(document_html)
    if display:
        print('\nFor Original:')
    avg, sets, sort, _, ans, pred, _ = compare(qtext, document_text, sa, display) #['Average Score', 'Set Score', 'Sort Score','Question', 'Answer', 'Predicted', 'Length of Text'])
    if ans not in ret[0]:
        ret[0] = ret[0] + ans + " -- "    
    ret.extend([avg, pred])
    col_names.append("Avg Score: original")
    col_names.append("Prediction: original")     
    
    all_avg = 0
    i = 0
    l_doc = []
    l_doc.append(document_text)
    for d in dates:
        if display:
            print('\nFor date: ', d)
        doctext, _ = get_wikipedia_revision(title, d, diagnostic = False)
        doctext = get_paragraphs(doctext)
        avg = 0
        pred = "No Edit"
        if doctext is not None:
            l_doc.append(doctext)
            avg, sets, sort, _, ans, pred, _ = compare(qtext, doctext, sa, display) #['Average Score', 'Set Score', 'Sort Score','Question', 'Answer', 'Predicted', 'Length of Text'])
            if ans not in ret[0]:
                ret[0] = ret[0] + ans + " -- "
            i = i + 1   
        ret.extend([avg, pred])
        all_avg += avg
        col_names.append("Avg Score: " + str(d))
        col_names.append("Predicted: " + str(d))
        
    if i == 0: 
        ret.insert(0, np.nan)
    else:
        ret.insert(0, all_avg/i)
        
    col_names.insert(0, "Overall Avg")
    return ret, col_names, l_doc


def compare(qtext, doctext, short_answers, display = False):

    question_text = qtext
    if display:
        print('Question: ', question_text)
    doc = doctext
    q = question_text
    answer = model.predict(doc,q)
    if display:
        print('Predicted Answer: ', answer['answer'])
    
    set_scores = [-1]
    sort_scores = [-1]
    all_avg_scores = [-1]
    for sa in short_answers:
        token_set_ratio = fuzz.token_set_ratio(sa, answer['answer'])
        token_sort_ratio = fuzz.token_sort_ratio(sa, answer['answer'])
        avg_score = (token_set_ratio + token_sort_ratio) / 2
        if display:
            print('Correct Answer: ', sa)
            print('Fuzzy score: ', avg_score)
        set_scores.append(token_set_ratio)
        sort_scores.append(token_sort_ratio)
        all_avg_scores.append(avg_score)
    if len(short_answers) == 0:
        short_answers = [None]
    max_index = all_avg_scores.index(max(all_avg_scores)) - 1
    return (max(all_avg_scores), max(set_scores), max(sort_scores), question_text, list(short_answers)[max_index], answer['answer'], len(doctext))


In [ ]:
# check how many have no short answers
# ind = []
# for i in range(len(list_of_q)):
#     l = list_of_q[i]
#     x = json.loads(l)
#     short_ans = get_short_answers(x)
#     if len(short_ans) != 0:
#         ind.append(i)


ind = []
for i in tqdm(range(len(list_of_q))):
    l = list_of_q[i]
    x = json.loads(l)
    s_ans = get_short_answers_train_set(x)
    if s_ans != set():
        ind.append(i)

In [ ]:
len(ind)

In [ ]:
# ret, col_names, l_doc = get_all_revs(list_of_q[2], display = True)
x = json.loads(list_of_q[ind[4]])
get_short_answers_train_set(x)

In [ ]:
# rows = []
# for q in tqdm(list_of_q):
#     try:
#         row, cols, l_doc = get_all_revs(q)
#     except:
#         continue
#     if cols is not None:
#         good_cols = cols
#     if row is not None:
#         rows.append(row)
        
    
# scoress = pd.DataFrame(rows, columns = good_cols).sort_values(['Overall Avg'], ascending = False)


rows = []
for i in tqdm(ind[0:20000]):
    try:
        row, cols, l_doc = get_all_revs(list_of_q[i])
    except:
        break
    if cols is not None:
        good_cols = cols
    if row is not None:
        rows.append(row)
        
    
scoress = pd.DataFrame(rows, columns = good_cols).sort_values(['Overall Avg'], ascending = False)


In [ ]:
scoress.shape

In [ ]:
scoress.to_csv('scores_short_answer_only_0_20k_new_dates.csv')

In [ ]:
# t,u = get_wikipedia_revision('Benjamin_Stone_(Law_&_Order_character)', datetime.datetime.now(), diagnostic = True)


scoress = pd.read_csv('scores_short_answer_only_10k.csv')

In [ ]:
# get_paragraphs(t)
# print(t)

In [ ]:
# !curl -X GET https://en.wikipedia.org/w/api.php?action=query&prop=revisions&titles=Therefore_sign&rvprop=ids%7Ctimestamp%7Ccomment&rvslots=main&rvstart=2021-04-21T22%3A29%3A08.802494&rvlimit=1&formatversion=2&format=json
    
    
scoress.shape

In [ ]:
# datetime.datetime(2013,1,1)

In [ ]:
scoress[scoress['Avg Score: original'] > 70].sample(10)

In [ ]:
scoress[scoress['Avg Score: original'] > 70].sample(10)

In [ ]:
scoress[scoress['Avg Score: original'] > 70].sample(10)

In [ ]:
scoress.shape

In [ ]:
scoress[scoress.isin(['No Edit']).any(axis=1)].shape

In [ ]:
scoress[scoress.isin(['Redirect']).any(axis=1)].shape

In [ ]:
scoress[~scoress.isin(['Redirect', 'No Edit']).any(axis=1)].shape

In [ ]:
s = scoress[~scoress.isin(['Redirect', 'No Edit']).any(axis=1)]
s = s[s['Avg Score: original'] > 70]
s.shape

In [ ]:
rows = []
for i in list_of_q:
    x = json.loads(i)
    rows.append([x["question_text"], x["document_title"]])
    
titles = pd.DataFrame(rows, columns = ["Question", "Wiki Page"])


In [ ]:
s